<a href="https://colab.research.google.com/github/yingtongxiong/MLCnotebooks/blob/xytExercise/2_4_TensorIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python3 -m  pip install mlc-ai-nightly -f https://mlc.ai/wheels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://mlc.ai/wheels
     |████████████████████████████████| 45.5 MB 1.5 MB/s 


In [2]:
import numpy as np
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T

In [3]:
dtype = "float32"
a_np = np.random.rand(128, 128).astype(dtype)
b_np = np.random.rand(128, 128).astype(dtype)
# a @ b is equal to np.matmul(a, b)
c_mm_relu = np.maximum(a_np @ b_np, 0)

In [4]:
def lnumpy_mm_relu(A: np.ndarray, B: np.ndarray, C: np.ndarray):
  Y = np.empty((128, 128), dtype="float32")
  for i in range(128):
    for j in range(128):
      for k in range(128):
        if k == 0:
          Y[i, j] = 0
        Y[i, j] = Y[i, j] + A[i, k] * B[k, j]
  for i in range(128):
    for j in range(128):
      C[i, j] = max(Y[i, j], 0)

In [5]:
c_np = np.empty((128, 128), dtype=dtype)
lnumpy_mm_relu(a_np, b_np, c_np)
np.testing.assert_allclose(c_mm_relu, c_np, rtol=1e-5)